In [ ]:
data = data[data["rating"] != "Puan yok"]

# Temizlenen veri setini kaydetme
data.to_csv("temiz_veri_son.csv", index=False)

print(f"Temizlenen veri satır sayısı: {len(data)}")

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import matplotlib.pyplot as plt
import seaborn as sns

# Veri yükleme ve hazırlama
def load_and_prepare_data(data):
    # DataFrame oluşturma
    df = pd.DataFrame(data, columns=['kullanici_adi', 'kitap_adi', 'puan'])
    
    # Veri tipleri dönüşümü
    df['puan'] = df['puan'].astype(float)
    
    # Kategorik değişkenleri sayısal değerlere dönüştürme
    le_users = LabelEncoder()
    le_books = LabelEncoder()
    
    df['kullanici_id'] = le_users.fit_transform(df['kullanici_adi'])
    df['kitap_id'] = le_books.fit_transform(df['kitap_adi'])
    
    return df

# Veri analizi fonksiyonu
def analyze_data(df):
    print("\nVeri Analizi:")
    print("-------------")
    print(f"Toplam değerlendirme sayısı: {len(df)}")

    print("\nPuan istatistikleri:")
    print(df['puan'].describe())
    
    # Kullanıcı bazlı ortalama puanlar
    user_ratings = df.groupby('kullanici_adi')['puan'].agg(['mean', 'count'])
    print("\nKullanıcı bazlı ortalama puanlar:")
    print(user_ratings)

# Model eğitim ve değerlendirme fonksiyonu
def train_and_evaluate_models(df):
    # Feature'ları ve target'ı ayırma
    X = df[['kullanici_id', 'kitap_id']]
    y = df['puan']
    
    # Veriyi train ve test olarak bölme
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Modeller
    models = {
        'KNN': KNeighborsRegressor(n_neighbors=3),
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
        'SVR': SVR(kernel='rbf')
    }
    
    results = {}
    
    for name, model in models.items():
        # Model eğitimi
        model.fit(X_train, y_train)
        
        # Tahminler
        y_pred = model.predict(X_test)
        
        # Metrikler
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results[name] = {
            'MSE': mse,
            'MAE': mae,
            'R2': r2
        }
    
    return results

# Örnek veri
data = pd.read_csv("temiz_veri_son.csv", names=["kullanici_id", "kitap_id", "puan"])

# Ana analiz akışı
df = load_and_prepare_data(data)
analyze_data(df)
results = train_and_evaluate_models(df)

# Sonuçları yazdırma
print("\nModel Performans Sonuçları:")
print("---------------------------")
for model_name, metrics in results.items():
    print(f"\n{model_name}:")
    for metric_name, value in metrics.items():
        print(f"{metric_name}: {value:.4f}")


Veri Analizi:
-------------
Toplam değerlendirme sayısı: 16641
Unique kullanıcı sayısı: 0
Unique kitap sayısı: 0

Puan istatistikleri:
count    16641.000000
mean         8.262124
std          1.638308
min          1.000000
25%          8.000000
50%          8.000000
75%         10.000000
max         10.000000
Name: puan, dtype: float64

Kullanıcı bazlı ortalama puanlar:
Empty DataFrame
Columns: [mean, count]
Index: []

Model Performans Sonuçları:
---------------------------

KNN:
MSE: 3.2508
MAE: 1.2881
R2: -0.2152

Random Forest:
MSE: 2.6759
MAE: 1.2413
R2: -0.0003

SVR:
MSE: 2.6950
MAE: 1.2305
R2: -0.0075
